# Hands On Decision Trees and Random Forests

In [1]:
# Import Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [2]:
# import Titanic Data from seaborn
Titanic = sns.load_dataset('titanic')

In [3]:
Titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Data Wrangling

In [4]:
Titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [5]:
# drop missing data
Titanic.dropna(inplace=True)

In [6]:
# drop redundant columns
Titanic.drop(['class', 'who', 'adult_male', 'embark_town', 'alone', 'alive'], axis=1, inplace=True)
Titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
1,1,1,female,38.0,1,0,71.2833,C,C
3,1,1,female,35.0,1,0,53.1000,S,C
6,0,1,male,54.0,0,0,51.8625,S,E
10,1,3,female,4.0,1,1,16.7000,S,G
11,1,1,female,58.0,0,0,26.5500,S,C


In [7]:
Titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 1 to 889
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   survived  182 non-null    int64   
 1   pclass    182 non-null    int64   
 2   sex       182 non-null    object  
 3   age       182 non-null    float64 
 4   sibsp     182 non-null    int64   
 5   parch     182 non-null    int64   
 6   fare      182 non-null    float64 
 7   embarked  182 non-null    object  
 8   deck      182 non-null    category
dtypes: category(1), float64(2), int64(4), object(2)
memory usage: 13.3+ KB


#### convert any floats to integers

In [8]:
Titanic.age = Titanic.age.astype(int)

In [9]:
Titanic.fare = Titanic.fare.astype(int)

In [10]:
Titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 1 to 889
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   survived  182 non-null    int64   
 1   pclass    182 non-null    int64   
 2   sex       182 non-null    object  
 3   age       182 non-null    int32   
 4   sibsp     182 non-null    int64   
 5   parch     182 non-null    int64   
 6   fare      182 non-null    int32   
 7   embarked  182 non-null    object  
 8   deck      182 non-null    category
dtypes: category(1), int32(2), int64(4), object(2)
memory usage: 11.9+ KB


#### recode columns where necessary

In [11]:
Titanic.deck.value_counts()

C    51
B    43
D    31
E    30
A    12
F    11
G     4
Name: deck, dtype: int64

In [12]:
def deckRecode (series):
    if series == 'A':
        return 0
    if series == 'B':
        return 1
    if series == 'C':
        return 2
    if series == 'D':
        return 3
    if series == 'E':
        return 4
    if series == 'F':
        return 5
    if series == 'G':
        return 6

Titanic['deckR'] = Titanic['deck'].apply(deckRecode)
Titanic.drop('deck', axis = 1, inplace = True)
Titanic.deckR = Titanic.deckR.astype(int)

In [ ]:
Titanic.sex.value_counts()

In [13]:
def sexRecode (series):
    if series == "female":
        return 0
    if series == "male":
        return 1
Titanic['sexR'] = Titanic['sex'].apply(sexRecode)
Titanic.drop('sex', axis = 1, inplace = True)
Titanic.sexR = Titanic.sexR.astype(int)

In [ ]:
Titanic.embarked.value_counts()

In [14]:
def embarkedRecode (series):
    if series == "S":
        return 0
    if series == "C":
        return 1
    if series == "Q":
        return 2
Titanic['embarkedR'] = Titanic['embarked'].apply(embarkedRecode)
Titanic.drop('embarked', axis = 1, inplace = True)
Titanic.embarkedR = Titanic.embarkedR.astype(int)

In [15]:
Titanic.head()

,survived,pclass,age,sibsp,parch,fare,deckR,sexR,embarkedR
1,1,1,38,1,0,71,2,0,1
3,1,1,35,1,0,53,2,0,0
6,0,1,54,0,0,51,4,1,0
10,1,3,4,1,1,16,6,0,0
11,1,1,58,0,0,26,2,0,0


## Decision Tree

#### Specify X and Y

In [16]:
x = Titanic.drop('survived', axis = 1)
y = Titanic[['survived']]

#### Train Test Split

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state = 76)

#### Create initial decison tree

In [18]:
decisionTree = DecisionTreeClassifier(random_state = 76)
decisionTree.fit(x_train, y_train)

DecisionTreeClassifier(random_state=76)

#### Assess the model

In [19]:
treePredictions = decisionTree.predict(x_test)
print(confusion_matrix(y_test, treePredictions))
print(classification_report(y_test, treePredictions))

[[17  2]
 [ 9 27]]
              precision    recall  f1-score   support

           0       0.65      0.89      0.76        19
           1       0.93      0.75      0.83        36

    accuracy                           0.80        55
   macro avg       0.79      0.82      0.79        55
weighted avg       0.84      0.80      0.80        55



### This model is 84% accurate. It pedicts death correctly 65% of the time and survival 93% of the time.

# Part II Random Forest

In [25]:
forest = RandomForestClassifier(n_estimators = 500, random_state = 76)
forest.fit(x_train, y_train)

C:\Users\dez_g\AppData\Local\Temp\ipykernel_6012\415135486.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(x_train, y_train)


RandomForestClassifier(n_estimators=500, random_state=76)

#### Evaluate Model Fit

In [26]:
forestPredictions = forest.predict(x_test)
print(confusion_matrix(y_test, forestPredictions))
print(classification_report(y_test, forestPredictions))

[[13  6]
 [ 7 29]]
              precision    recall  f1-score   support

           0       0.65      0.68      0.67        19
           1       0.83      0.81      0.82        36

    accuracy                           0.76        55
   macro avg       0.74      0.74      0.74        55
weighted avg       0.77      0.76      0.77        55



#### this random forest model is less accurate (somehow) at 77% than the decision tree model at 84%.